In [1]:
# Stopwords

english_stopwords = stopwords.words('english')
print(f"Length of stopwords set before filtering: {len(english_stopwords)}")

"""
    Filter a set of stopwords by removing words which match certain regular expressions.
    Used to filter out stopwords indicating nefgation (e.g. "can't").
"""
def filter_stopwords(stopwords, patterns):                  
    non_negated_stopwords = set(
        [stopword for stopword in stopwords
        if not any(re.search(pattern, stopword, re.IGNORECASE) for pattern in patterns)
        ])
    return non_negated_stopwords

non_negated_english_stopwords = filter_stopwords(english_stopwords, negation_patterns)
print(f"Length of stopwords set after filtering: {len(non_negated_english_stopwords)}")

## Now get the negated tokens (from using WordNet negation handler) and remove the stopwords from the tokens

"""
    Filters stopwords from a list of tokens
"""
def remove_stopwords_from_tokens(tokens, stopwords):
    new_tokens = [token for token in tokens if token not in stopwords]
    return new_tokens

NameError: name 'stopwords' is not defined

In [2]:
# Get the tokens after applying the new negation handling function
wordnet_negated_train_tokens_no_stopwords = [
    remove_stopwords_from_tokens(tokens, non_negated_english_stopwords)
    for tokens in wordnet_negated_train_tokens
]
wordnet_negated_validation_tokens_no_stopwords = [
    remove_stopwords_from_tokens(tokens, non_negated_english_stopwords)
    for tokens in wordnet_negated_validation_tokens
]

NameError: name 'wordnet_negated_train_tokens' is not defined